# Audio Welding Defect Detection — Unified Training Notebook

**Single notebook for the full pipeline: config → train → eval → sweep → export .pt**

## Workflow
1. **Cell 1 — Setup**: run once, sets up paths and helpers
2. **Cell 2 — Config**: edit your parameters here, then run
3. **Cell 3 — Data stats**: optional sanity check on the dataset
4. **Cell 4 — Train**: reset + train from scratch (or resume)
5. **Cell 5 — Evaluate**: load best checkpoint and print metrics
6. **Cell 6 — Hyperparameter sweep**: optional grid search
7. **Cell 7 — Export .pt**: build deployable TorchScript model
8. **Cell 8 — Smoke-test .pt**: verify exported model on a sample file

## Training philosophy (always MIL)

For every audio file, the model predicts a probability vector for each sliding window.
Those per-window predictions are aggregated (top-k pooling) into a single bag-level
prediction, and the loss is computed on that — never on individual windows.

| `TASK`         | Top-k selection | Loss | Deploy class | Output |
|----------------|-----------------|------|--------------|--------|
| `"binary"`     | top-k by P(defect), asymmetric ratios | BCE | `DeploySingleLabelMIL` | `{label, p_defect}` |
| `"multiclass"` | top-k by P(correct class) → avg full prob vector | NLL | `DeployMulticlassFile` | `{label, probs[7]}` |

## Two inference modes (both from the .pt)
- `model(waveform)` — full audio → chunks → aggregate → prediction
- `model.predict_window(window)` — single pre-cut chunk → direct prediction

In [1]:
# ── Cell 1: Setup ─────────────────────────────────────────────────────────────
# Run once. Nothing to edit here.

import json, os, sys, shlex, subprocess, shutil, copy, itertools, select, pty
from pathlib import Path

import torch

PROJECT_ROOT = Path.cwd()
PYTHON       = sys.executable
CONFIG_PATH  = PROJECT_ROOT / "configs" / "master_config.json"

sys.path.insert(0, str(PROJECT_ROOT))

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Python      : {PYTHON}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Device      : {device}")


def _stream(cmd, cwd=None):
    """Run a command inside a pseudo-TTY so tqdm renders in-place (no line spam)."""
    cmd = [str(c) for c in cmd]
    print("$", " ".join(shlex.quote(c) for c in cmd), flush=True)
    env = {**os.environ, "PYTHONUNBUFFERED": "1"}

    # Open a master/slave PTY pair; give the slave end to the child process so
    # tqdm detects a real terminal and uses \r instead of \n for updates.
    master, slave = pty.openpty()
    p = subprocess.Popen(
        cmd, cwd=str(cwd or PROJECT_ROOT), env=env,
        stdin=slave, stdout=slave, stderr=slave,
        close_fds=True,
    )
    os.close(slave)

    # Stream output from the master end of the PTY to the notebook cell.
    # \r from tqdm will overwrite the current line in Jupyter/VSCode output.
    while p.poll() is None:
        try:
            r, _, _ = select.select([master], [], [], 0.05)
        except (ValueError, select.error):
            break
        if r:
            try:
                data = os.read(master, 4096)
            except OSError:
                break
            sys.stdout.write(data.decode("utf-8", errors="replace"))
            sys.stdout.flush()

    # Drain any remaining bytes after the process exits.
    try:
        while True:
            r, _, _ = select.select([master], [], [], 0.1)
            if not r:
                break
            data = os.read(master, 4096)
            if not data:
                break
            sys.stdout.write(data.decode("utf-8", errors="replace"))
            sys.stdout.flush()
    except OSError:
        pass

    try:
        os.close(master)
    except OSError:
        pass

    rc = p.wait()
    print(f"\n[exit {rc}]", flush=True)
    if rc != 0:
        raise subprocess.CalledProcessError(rc, cmd)


def _write_config(cfg: dict):
    CONFIG_PATH.parent.mkdir(parents=True, exist_ok=True)
    with open(CONFIG_PATH, "w") as f:
        json.dump(cfg, f, indent=2)


print("Setup complete.")

Python      : /home/alolli/miniconda3/envs/therness_env/bin/python
Project root: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito
Device      : cuda
Setup complete.


In [12]:
# ── Cell 2: Configuration ─────────────────────────────────────────────────────
# ★ EDIT THIS CELL, then run cells 3-8 in order.

# ─────────────────────────────────────────────
# TASK SELECTION
#   "binary"      → good_weld vs defect (2 classes)
#   "multiclass"  → per-defect-type (7 classes)
# ─────────────────────────────────────────────
TASK = "multiclass"       # "binary"  |  "multiclass"

# ─────────────────────────────────────────────
# PATHS
# ─────────────────────────────────────────────
DATA_ROOT = "/data1/malto/therness/data/Hackathon"
CKPT_DIR  = f"checkpoints/audio_{TASK}"   # separate dirs per task

# ─────────────────────────────────────────────
# TRAINING BASICS
# ─────────────────────────────────────────────
NUM_EPOCHS       = 70      # extended training budget
LR               = 7e-5    # strong multiclass baseline
WEIGHT_DECAY     = 3e-5    # promoted from best research trial
PATIENCE         = 18      # allow longer learning before early stop
TRAIN_FRACTION   = 1.0     # fraction of files to use; < 1.0 for fast iteration
VAL_SPLIT        = 0.2
SEED             = 42
NUM_WORKERS      = 4

# ─────────────────────────────────────────────
# MIL SETTINGS
#
# Training always uses MIL: slide windows over the file, predict per window,
# aggregate via top-k pooling, compute loss on the file-level prediction.
# ─────────────────────────────────────────────

MIL_BATCH_SIZE = 8   # files per gradient step

# Top-k selection ratios
TOPK_RATIO_POS  = 0.12
TOPK_RATIO_NEG  = 0.20
EVAL_POOL_RATIO = TOPK_RATIO_POS   # keep eval aligned with positive pooling

# Auxiliary per-window regulariser on good_weld files (binary only).
# Keep disabled by default to avoid biasing training toward all-good predictions.
GOOD_WINDOW_WEIGHT = 0.0

# After training, sweep threshold on val set and pick the one maximising F1.
# Set False to keep a fixed 0.5.
AUTO_THRESHOLD = True

# Multiclass-specific imbalance controls (safe defaults for binary too).
USE_CLASS_WEIGHTS      = True
CLASS_WEIGHT_POWER     = 0.65
MULTICLASS_EVAL_MODE   = "topk_per_class"
USE_BALANCED_SAMPLER   = True
BALANCED_SAMPLER_POWER = 0.35

# ─────────────────────────────────────────────
# AUDIO FEATURES (rarely need changing)
# ─────────────────────────────────────────────
SAMPLING_RATE  = 16000
CHUNK_LENGTH_S = 1.0     # seconds
N_MELS         = 40
DROPOUT        = 0.15

# ──────────────────────────────────────────────────────────────
# Build audio config dict
# ──────────────────────────────────────────────────────────────
_AUDIO_CFG = {
    "project_name": "therness-welding-hackathon",
    "data_root": DATA_ROOT,
    "num_classes": 2 if TASK == "binary" else 7,
    "device": "auto",
    "audio": {
        "feature_params": {
            "sampling_rate": SAMPLING_RATE,
            "n_fft": 1024,
            "frame_length_in_s": 0.04,
            "frame_step_in_s": 0.02,
            "n_mels": N_MELS,
            "f_min": 0,
            "f_max": 8000,
            "chunk_length_in_s": CHUNK_LENGTH_S,
            "normalize": True,
        },
        "model": {"dropout": DROPOUT},
        "training": {
            "train_fraction": TRAIN_FRACTION,
            "num_epochs": NUM_EPOCHS,
            "lr": LR,
            "weight_decay": WEIGHT_DECAY,
            "lr_schedule": {
                "warmup_ratio": 0.1,
                "plateau_factor": 0.5,
                "plateau_patience": 4,
                "plateau_threshold": 1e-3,
                "plateau_min_lr": 1e-6,
            },
            "patience": PATIENCE,
            "val_split": VAL_SPLIT,
            "seed": SEED,
            "num_workers": NUM_WORKERS,
            "task": TASK,
            "sequence_mil": {
                "enabled": True,            # always on — no chunk-level CE mode
                "batch_size": MIL_BATCH_SIZE,
                "topk_ratio_pos": TOPK_RATIO_POS,
                "topk_ratio_neg": TOPK_RATIO_NEG,
                "eval_pool_ratio": EVAL_POOL_RATIO,
                "auto_threshold": AUTO_THRESHOLD,
                "threshold": 0.5,
                "good_window_weight": GOOD_WINDOW_WEIGHT,
                "use_class_weights": USE_CLASS_WEIGHTS,
                "class_weight_power": CLASS_WEIGHT_POWER,
                "multiclass_eval_mode": MULTICLASS_EVAL_MODE,
                "use_balanced_sampler": USE_BALANCED_SAMPLER,
                "balanced_sampler_power": BALANCED_SAMPLER_POWER,
            },
            "metric": "macro_f1",
            "checkpoint_dir": CKPT_DIR,
        },
    },
}

# ── Merge: preserve video (and any other) sections already in the file ─────────
_existing = {}
if CONFIG_PATH.exists():
    with open(CONFIG_PATH) as _f:
        _existing = json.load(_f)
_existing.update(_AUDIO_CFG)
CFG = _existing
_write_config(CFG)

print(f"{'═'*60}")
print(f"  TASK             : {TASK}")
print(f"  Epochs / Patience: {NUM_EPOCHS} / {PATIENCE}")
print(f"  LR               : {LR}")
print(f"  MIL batch size   : {MIL_BATCH_SIZE} files/batch")
print(f"  Top-k pos / neg  : {TOPK_RATIO_POS} / {TOPK_RATIO_NEG}  (eval={EVAL_POOL_RATIO})")
print(f"  Good win weight  : {GOOD_WINDOW_WEIGHT}")
print(f"  Class weighting  : {USE_CLASS_WEIGHTS} (power={CLASS_WEIGHT_POWER})")
print(f"  MC eval mode     : {MULTICLASS_EVAL_MODE}")
print(f"  Balanced sampler : {USE_BALANCED_SAMPLER} (power={BALANCED_SAMPLER_POWER})")
print(f"  Chunk length (s) : {CHUNK_LENGTH_S}")
print(f"  Train fraction   : {TRAIN_FRACTION}")
print(f"  Checkpoint dir   : {CKPT_DIR}")
print(f"  Config written   : {CONFIG_PATH}")
print(f"  Video section    : {'preserved ✓' if 'video' in CFG else 'not present'}")
print(f"{'═'*60}")

════════════════════════════════════════════════════════════
  TASK             : multiclass
  Epochs / Patience: 70 / 18
  LR               : 7e-05
  MIL batch size   : 8 files/batch
  Top-k pos / neg  : 0.12 / 0.2  (eval=0.12)
  Good win weight  : 0.0
  Class weighting  : True (power=0.65)
  MC eval mode     : topk_per_class
  Balanced sampler : True (power=0.35)
  Chunk length (s) : 1.0
  Train fraction   : 1.0
  Checkpoint dir   : checkpoints/audio_multiclass
  Config written   : /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.json
  Video section    : preserved ✓
════════════════════════════════════════════════════════════


In [6]:
# ── Cell 3: Data Stats (optional sanity check) ────────────────────────────────
# Quick look at how many files exist and their class distribution.
# Safe to skip — training will print the same info.

import glob, re
from collections import Counter

_DEFECT_RE = re.compile(r"^(?P<defect>.+?)(?:_weld)?_\d+_")

def _label(path, data_root, task):
    rel   = os.path.relpath(path, data_root)
    parts = rel.split(os.sep)
    if parts[0] == "good_weld":
        defect = "good_weld"
    else:
        folder = parts[1] if len(parts) > 1 else ""
        m      = _DEFECT_RE.match(folder)
        defect = m.group("defect") if m else folder
    if task == "binary":
        return "good_weld" if defect == "good_weld" else "defect"
    return defect

all_files = sorted(glob.glob(os.path.join(DATA_ROOT, "**", "*.flac"), recursive=True))
counts    = Counter(_label(f, DATA_ROOT, TASK) for f in all_files)

print(f"Total .flac files : {len(all_files)}")
print(f"Task              : {TASK}")
print(f"Label distribution:")
for label, n in sorted(counts.items()):
    pct = 100.0 * n / len(all_files)
    print(f"  {label:30s}  {n:5d}  ({pct:.1f}%)")

Total .flac files : 1551
Task              : multiclass
Label distribution:
  burnthrough                       169  (10.9%)
  crater_cracks                      75  (4.8%)
  excessive_convexity                80  (5.2%)
  excessive_penetration             259  (16.7%)
  good_weld                         731  (47.1%)
  lack_of_fusion                    158  (10.2%)
  overlap                            79  (5.1%)


In [13]:
# ── Cell 4: Train ─────────────────────────────────────────────────────────────
# Set RESET = True to wipe the checkpoint dir and train from scratch.
# Set RESET = False to resume from an existing checkpoint (if one exists).

RESET = True

ckpt = Path(CKPT_DIR)
if RESET and ckpt.exists():
    shutil.rmtree(ckpt)
    print(f"Removed checkpoint dir: {ckpt.resolve()}")

_stream([PYTHON, "-u", "-m", "audio.run_audio", "--config", str(CONFIG_PATH)])

Removed checkpoint dir: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/audio_multiclass
$ /home/alolli/miniconda3/envs/therness_env/bin/python -u -m audio.run_audio --config /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.json


Device: cuda

Total weld files: 1551
File label distribution (multiclass): {'excessive_penetration': 259, 'good_weld': 731, 'excessive_convexity': 80, 'lack_of_fusion': 158, 'crater_cracks': 75, 'burnthrough': 169, 'overlap': 79}
Split strategy: stratified
Train welds: 1240 | Val welds: 311
File split stats | train={ burnthrough: 135 (10.9%), crater_cracks: 60 (4.8%), excessive_convexity: 64 (5.2%), excessive_penetration: 207 (16.7%), good_weld: 585 (47.2%), lack_of_fusion: 126 (10.2%), overlap: 63 (5.1%) } | val={ burnthrough: 34 (10.9%), crater_cracks: 15 (4.8%), excessive_convexity: 16 (5.1%), excessive_penetration: 52 (16.7%), good_weld: 146 (46.9%), lack_of_fusion: 32 (10.3%), overlap: 16 (5.1%) }
Train files: 1240 | Val files: 311
Classes (7): {'burnthrough': 0, 'crater_cracks': 1, 'excessive_convexity': 2, 'excessive_penetration': 3, 'good_weld': 4, 'lack_of_fusion': 5, 'overlap': 6}
Multiclass balanced sampler enabled | power=0.35
DataLoader stats | train_batches=155 | val_batc

In [14]:
# ── Cell 5: Evaluate Best Checkpoint ─────────────────────────────────────────
# Loads best_model.pt and runs the full validation loop.

BEST_CKPT = Path(CKPT_DIR) / "best_model.pt"

if not BEST_CKPT.exists():
    raise FileNotFoundError(
        f"No best_model.pt found in '{CKPT_DIR}'.\n"
        "Run Cell 4 (Train) first."
    )

_stream([
    PYTHON, "-u", "-m", "audio.run_audio",
    "--config",     str(CONFIG_PATH),
    "--test_only",
    "--checkpoint", str(BEST_CKPT),
])

# ── Print checkpoint summary ──────────────────────────────────────────────────
try:
    ck = torch.load(str(BEST_CKPT), map_location="cpu", weights_only=True)
except TypeError:
    ck = torch.load(str(BEST_CKPT), map_location="cpu")

print()
print("─" * 50)
print("Best checkpoint summary")
print("─" * 50)
print(f"  Epoch     : {ck.get('epoch')}")
print(f"  Val F1    : {ck.get('val_f1', float('nan')):.4f}")
auc = ck.get('val_auc')
if auc is not None:
    print(f"  Val AUC   : {auc:.4f}  (binary only)")
thr = ck.get('threshold')
if thr is not None:
    print(f"  Threshold : {thr:.2f}   (binary MIL only)")
print(f"  Val loss  : {ck.get('val_loss', float('nan')):.4f}")
print(f"  File      : {BEST_CKPT.resolve()}")

# ── Decision gate (task-aware) ───────────────────────────────────────────────
print()
val_f1 = float(ck.get('val_f1', 0.0))
F1_MIN = 0.70 if TASK == "binary" else 0.50
if val_f1 >= F1_MIN:
    print(f"✓ Val F1 = {val_f1:.4f} ≥ {F1_MIN:.2f} for task='{TASK}' — model is acceptable, proceed to export.")
else:
    print(f"✗ Val F1 = {val_f1:.4f} < {F1_MIN:.2f} for task='{TASK}' — run Cell 6 sweep or improve data balance.")

$ /home/alolli/miniconda3/envs/therness_env/bin/python -u -m audio.run_audio --config /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.json --test_only --checkpoint checkpoints/audio_multiclass/best_model.pt
Device: cuda

Total weld files: 1551
File label distribution (multiclass): {'excessive_penetration': 259, 'good_weld': 731, 'excessive_convexity': 80, 'lack_of_fusion': 158, 'crater_cracks': 75, 'burnthrough': 169, 'overlap': 79}
Split strategy: stratified
Train welds: 1240 | Val welds: 311
File split stats | train={ burnthrough: 135 (10.9%), crater_cracks: 60 (4.8%), excessive_convexity: 64 (5.2%), excessive_penetration: 207 (16.7%), good_weld: 585 (47.2%), lack_of_fusion: 126 (10.2%), overlap: 63 (5.1%) } | val={ burnthrough: 34 (10.9%), crater_cracks: 15 (4.8%), excessive_convexity: 16 (5.1%), excessive_penetration: 52 (16.7%), good_weld: 146 (46.9%), lack_of_fusion: 32 (10.3%), overlap: 16 (5.1%) }
Train files: 1240 | Val files: 311
Classes (

In [ ]:
# ── Cell 6: Hyperparameter Research + Optional Final 100-epoch Train ──────────
# Fast controls to avoid very long runs:
#   1) run quick research shortlist
#   2) only then launch final 100 epochs manually

RUN_RESEARCH = True
RUN_FINAL_100 = True       # enabled: launch final run after selecting best resumed trial
FINAL_EPOCHS = 100
RESET_FINAL_CKPT = True

FAST_MODE = True           # True = much shorter research cycle
MAX_TRIALS = 6             # used only in FAST_MODE
RESUME_RESEARCH = True     # reuse completed trial checkpoints if available

if RUN_RESEARCH:
    if TASK == "multiclass":
        RESEARCH_EPOCHS = 12 if FAST_MODE else 30
        RESEARCH_PATIENCE = 4 if FAST_MODE else 10

        # Focused around your strongest region from previous runs
        research_candidates = [
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 6e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 8e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 3e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 7e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.11, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.13, "class_weight_power": 0.65, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.70, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.60, "balanced_sampler_power": 0.35},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.30},
            {"lr": 7e-5, "weight_decay": 5e-5, "topk_ratio_pos": 0.12, "class_weight_power": 0.65, "balanced_sampler_power": 0.40},
            {"lr": 8e-5, "weight_decay": 3e-5, "topk_ratio_pos": 0.13, "class_weight_power": 0.70, "balanced_sampler_power": 0.35},
        ]

        if FAST_MODE:
            research_candidates = research_candidates[:MAX_TRIALS]

        print(f"Task={TASK} | Research sweep: {len(research_candidates)} trials × {RESEARCH_EPOCHS} epochs")
        print(f"FAST_MODE={FAST_MODE} | RUN_FINAL_100={RUN_FINAL_100} | RESUME_RESEARCH={RESUME_RESEARCH}\n")

        def _run_trial(params, epochs, patience, prefix, idx, total):
            tag = (
                f"lr={params['lr']}_wd={params['weight_decay']}"
                f"_kp={params['topk_ratio_pos']}_cwp={params['class_weight_power']}"
                f"_bsp={params['balanced_sampler_power']}"
            )
            ckpt_dir = f"checkpoints/{prefix}_{TASK}_{tag}"
            best_pt = Path(ckpt_dir) / "best_model.pt"

            s_cfg = copy.deepcopy(CFG)
            t = s_cfg["audio"]["training"]
            m = t["sequence_mil"]

            t["num_epochs"] = epochs
            t["patience"] = patience
            t["checkpoint_dir"] = ckpt_dir
            t["lr"] = params["lr"]
            t["weight_decay"] = params["weight_decay"]

            m["topk_ratio_pos"] = params["topk_ratio_pos"]
            m["eval_pool_ratio"] = params["topk_ratio_pos"]
            m["use_class_weights"] = True
            m["class_weight_power"] = params["class_weight_power"]
            m["use_balanced_sampler"] = True
            m["balanced_sampler_power"] = params["balanced_sampler_power"]
            m["multiclass_eval_mode"] = "topk_per_class"

            _write_config(s_cfg)
            print(f"\n[{idx:02d}/{total:02d}] {params}")
            print(f"  → {ckpt_dir}")

            if RESUME_RESEARCH and best_pt.exists():
                print("  ↳ Reusing existing checkpoint (resume mode)")
                try:
                    bck = torch.load(str(best_pt), map_location="cpu", weights_only=True)
                except TypeError:
                    bck = torch.load(str(best_pt), map_location="cpu")
                return {
                    **params,
                    "val_f1": float(bck.get("val_f1", 0.0)),
                    "epoch": int(bck.get("epoch", -1)),
                    "ckpt": ckpt_dir,
                }

            try:
                _stream([PYTHON, "-u", "-m", "audio.run_audio", "--config", str(CONFIG_PATH)])
                try:
                    bck = torch.load(str(best_pt), map_location="cpu", weights_only=True)
                except TypeError:
                    bck = torch.load(str(best_pt), map_location="cpu")
                row = {
                    **params,
                    "val_f1": float(bck.get("val_f1", 0.0)),
                    "epoch": int(bck.get("epoch", -1)),
                    "ckpt": ckpt_dir,
                }
            except Exception as e:
                print(f"  FAILED: {e}")
                row = {**params, "val_f1": -1.0, "epoch": -1, "ckpt": ckpt_dir}
            return row

        research_results = []
        for i, params in enumerate(research_candidates, start=1):
            research_results.append(_run_trial(params, RESEARCH_EPOCHS, RESEARCH_PATIENCE, "research", i, len(research_candidates)))

        research_results.sort(key=lambda r: -r["val_f1"])

        print("\n" + "═" * 112)
        print(f"RESEARCH RESULTS — {TASK} — sorted by val_f1")
        print("═" * 112)
        for i, r in enumerate(research_results):
            marker = "★" if i == 0 else " "
            print(
                f"  {marker} F1={r['val_f1']:.4f} ep={r['epoch']:3d} "
                f"lr={r['lr']} wd={r['weight_decay']} kp={r['topk_ratio_pos']} "
                f"cwp={r['class_weight_power']} bsp={r['balanced_sampler_power']}"
            )
        print("═" * 112)

        if not research_results or research_results[0]["val_f1"] < 0:
            _write_config(CFG)
            raise RuntimeError("Research failed for all candidates; aborting final training.")

        best = research_results[0]
        print("\nBest research config:")
        print(
            f"  LR={best['lr']}, WEIGHT_DECAY={best['weight_decay']}, "
            f"TOPK_RATIO_POS={best['topk_ratio_pos']}, "
            f"CLASS_WEIGHT_POWER={best['class_weight_power']}, "
            f"BALANCED_SAMPLER_POWER={best['balanced_sampler_power']}"
        )

        if RUN_FINAL_100:
            print(f"\nStarting final training with best config for {FINAL_EPOCHS} epochs...")
            final_cfg = copy.deepcopy(CFG)
            t = final_cfg["audio"]["training"]
            m = t["sequence_mil"]

            t["num_epochs"] = FINAL_EPOCHS
            t["patience"] = max(PATIENCE, 25)
            t["lr"] = best["lr"]
            t["weight_decay"] = best["weight_decay"]
            t["checkpoint_dir"] = CKPT_DIR

            m["topk_ratio_pos"] = best["topk_ratio_pos"]
            m["eval_pool_ratio"] = best["topk_ratio_pos"]
            m["use_class_weights"] = True
            m["class_weight_power"] = best["class_weight_power"]
            m["use_balanced_sampler"] = True
            m["balanced_sampler_power"] = best["balanced_sampler_power"]
            m["multiclass_eval_mode"] = "topk_per_class"

            final_ckpt_dir = Path(CKPT_DIR)
            if RESET_FINAL_CKPT and final_ckpt_dir.exists():
                shutil.rmtree(final_ckpt_dir)
                print(f"Removed checkpoint dir: {final_ckpt_dir.resolve()}")

            _write_config(final_cfg)
            _stream([PYTHON, "-u", "-m", "audio.run_audio", "--config", str(CONFIG_PATH)])
            print("\nFinal 100-epoch training finished.")
            print("Run Cell 5 to evaluate and confirm the final checkpoint.")
        else:
            print("\nSkipped final 100-epoch run (RUN_FINAL_100=False).")
            print("When ready, set RUN_FINAL_100=True and re-run this cell.")

        _write_config(CFG)
        print("\nOriginal config restored.")

    else:
        print("This research cell is configured for multiclass only.")
else:
    print("RUN_RESEARCH = False — skipped.")

Task=multiclass | Research sweep: 6 trials × 12 epochs
FAST_MODE=True | RUN_FINAL_100=False | RESUME_RESEARCH=True


[01/06] {'lr': 7e-05, 'weight_decay': 5e-05, 'topk_ratio_pos': 0.12, 'class_weight_power': 0.65, 'balanced_sampler_power': 0.35}
  → checkpoints/research_multiclass_lr=7e-05_wd=5e-05_kp=0.12_cwp=0.65_bsp=0.35
  ↳ Reusing existing checkpoint (resume mode)

[02/06] {'lr': 6e-05, 'weight_decay': 5e-05, 'topk_ratio_pos': 0.12, 'class_weight_power': 0.65, 'balanced_sampler_power': 0.35}
  → checkpoints/research_multiclass_lr=6e-05_wd=5e-05_kp=0.12_cwp=0.65_bsp=0.35
  ↳ Reusing existing checkpoint (resume mode)

[03/06] {'lr': 8e-05, 'weight_decay': 5e-05, 'topk_ratio_pos': 0.12, 'class_weight_power': 0.65, 'balanced_sampler_power': 0.35}
  → checkpoints/research_multiclass_lr=8e-05_wd=5e-05_kp=0.12_cwp=0.65_bsp=0.35
  ↳ Reusing existing checkpoint (resume mode)

[04/06] {'lr': 7e-05, 'weight_decay': 3e-05, 'topk_ratio_pos': 0.12, 'class_weight_power': 0.65, 'balanced_sampler_

In [15]:
# ── Cell 7: Export .pt ───────────────────────────────────────────────────────
# Wraps the trained model in a self-contained TorchScript module.
#
# Binary  → DeploySingleLabelMIL  : model(waveform) → {label, p_defect}
# Multiclass → DeployMulticlassFile: model(waveform) → {label, probs[7]}
#
# Both expose:
#   model(waveform)              — full audio file, auto-chunked
#   model.predict_window(window) — single pre-cut chunk

BEST_CKPT = Path(CKPT_DIR) / "best_model.pt"
DEPLOY_PT = Path(CKPT_DIR) / f"deploy_{TASK}.pt"

if not BEST_CKPT.exists():
    raise FileNotFoundError(
        f"No best_model.pt found in '{CKPT_DIR}'.\n"
        "Run Cell 4 (Train) first."
    )

_stream([
    PYTHON, "-u", "-m", "audio.export_deploy_pt",
    "--checkpoint", str(BEST_CKPT),
    "--output",     str(DEPLOY_PT),
])

size_mb = DEPLOY_PT.stat().st_size / 1e6
print(f"\nDeploy artifact : {DEPLOY_PT.resolve()}")
print(f"File size       : {size_mb:.2f} MB")
print()
print("Usage in production:")
print("  import torch, torchaudio")
print(f"  model = torch.jit.load('{DEPLOY_PT.name}')")
print("  model.eval()")
print("  waveform, sr = torchaudio.load('weld.flac')  # shape (C, N)")
print("  out = model(waveform)            # full-file mode")
print("  out = model.predict_window(w)    # single-chunk mode")
if TASK == "binary":
    print("  # out['label']    → 0=good_weld, 1=defect")
    print("  # out['p_defect'] → probability of defect [0, 1]")
else:
    print("  # out['label'] → class index (0-6)")
    print("  # out['probs'] → softmax probabilities [7]")

$ /home/alolli/miniconda3/envs/therness_env/bin/python -u -m audio.export_deploy_pt --checkpoint checkpoints/audio_multiclass/best_model.pt --output checkpoints/audio_multiclass/deploy_multiclass.pt
Export mode : multiclass (DeployMulticlassFile)
  num_classes    = 7
  chunk_samples  = 16000  (1.0s @ 16000 Hz)

Saved: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/audio_multiclass/deploy_multiclass.pt
Methods available on loaded model:
  model(waveform)              → file-level prediction
  model.predict_window(window) → single-window prediction
  model.extract_window_activation(window)    → (128,) embedding
  model.extract_file_activations(waveform)   → (T, 128) embeddings
  model.extract_file_activation_mean(waveform) → (128,) embedding
  model.extract_window_activations(window)   → all stage/head activations
  model.extract_file_activation_summary(waveform) → mean stage/head activations

[exit 0]

Deploy artifact : /home/alolli/src/malto/hackathon/therne

In [51]:
# ── Cell 8: Smoke-test the exported .pt ──────────────────────────────────────
# Set SAMPLE_FILE to an absolute path of any .flac file to verify the export.
# The deployed model should accept any sample rate (resampled automatically here).

SAMPLE_FILE = ""   # ← e.g. "/data1/malto/therness/data/Hackathon/good_weld/good_weld_001_/audio.flac"

DEPLOY_PT = Path(CKPT_DIR) / f"deploy_{TASK}.pt"

if not SAMPLE_FILE:
    print("No SAMPLE_FILE set — skipping smoke-test.")
    print("Set SAMPLE_FILE to a .flac path and re-run this cell.")
elif not Path(SAMPLE_FILE).exists():
    print(f"File not found: {SAMPLE_FILE}")
else:
    import torchaudio

    # Load and resample to 16 kHz
    waveform, sr = torchaudio.load(SAMPLE_FILE)
    if sr != SAMPLING_RATE:
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
    print(f"Audio     : {Path(SAMPLE_FILE).name}")
    print(f"Shape     : {tuple(waveform.shape)}  ({waveform.shape[-1]/SAMPLING_RATE:.1f}s)")

    # Load deployed model
    try:
        dmodel = torch.jit.load(str(DEPLOY_PT), map_location="cpu")
    except Exception as e:
        raise RuntimeError(f"Failed to load {DEPLOY_PT}: {e}")
    dmodel.eval()

    # ── Full-file mode ────────────────────────────────────────────
    with torch.no_grad():
        file_out = dmodel(waveform)

    print()
    print("─── Full-file prediction ───")
    print(f"  label    : {file_out['label'].item()}", end="")
    if TASK == "binary":
        print(f"  (0=good_weld, 1=defect)")
        print(f"  p_defect : {file_out['p_defect'].item():.4f}")
    else:
        print()
        probs = file_out['probs'].tolist()
        print(f"  probs    : {[f'{p:.3f}' for p in probs]}")

    # ── Single-window mode ────────────────────────────────────────
    chunk_samples = int(CHUNK_LENGTH_S * SAMPLING_RATE)
    if waveform.shape[-1] >= chunk_samples:
        window = waveform[:1, :chunk_samples]   # mono, first chunk
        with torch.no_grad():
            win_out = dmodel.predict_window(window)
        print()
        print("─── Single-window prediction (first chunk) ───")
        print(f"  label    : {win_out['label'].item()}", end="")
        if TASK == "binary":
            print(f"  (0=good_weld, 1=defect)")
            print(f"  p_defect : {win_out['p_defect'].item():.4f}")
        else:
            print()
            win_probs = win_out['probs'].tolist()
            print(f"  probs    : {[f'{p:.3f}' for p in win_probs]}")
    else:
        print("(audio too short for single-window test)")

No SAMPLE_FILE set — skipping smoke-test.
Set SAMPLE_FILE to a .flac path and re-run this cell.
